In [2]:
#!pip install 'dlt[duckdb]'
#!pip install 'dlt[bigquery]'
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 35.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.2/731.2 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np
import json
from datetime import datetime
import duckdb

import pyarrow
import dlt
from dlt.sources.helpers.rest_client import RESTClient
from dlt.sources.helpers.rest_client.paginators import OffsetPaginator, BasePaginator

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [4]:
from typing import Any, List, Optional
from dlt.sources.helpers.rest_client.paginators import BasePaginator
from dlt.sources.helpers.requests import Response, Request

class QueryParamPaginator(BasePaginator):
    def __init__(self, page_param: str = "page", initial_page: int = 1):
        super().__init__()
        self.page_param = page_param
        self.page = initial_page

    def init_request(self, request: Request) -> None:
        # This will set the initial page number (e.g., page=1)
        self.update_request(request)

    def update_state(self, response: Response, data: Optional[List[Any]] = None) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.page += 1

    def update_request(self, request: Request) -> None:
        if request.params is None:
            request.params = {}
        request.params[self.page_param] = self.page

In [5]:
from typing import Any, List, Optional
from dlt.sources.helpers.rest_client.paginators import BasePaginator
from dlt.sources.helpers.requests import Response, Request

#source for custom paginator: https://dlthub.com/docs/general-usage/http/rest-client#paginators
#Chatgpt helped me build it based on that page.
class TimeRangePaginator(BasePaginator):
    def __init__(self, start_time: int, end_time: int, interval_ms: int):
        super().__init__()
        self.current_time = start_time
        self.end_time = end_time
        self.interval_ms = interval_ms

    def init_request(self, request: Request) -> None:
        """Initialize the request with the first startTime and endTime"""
        self.update_request(request)

    def update_state(self, response: Response, data: Optional[List[Any]] = None) -> None:
        """Update the pagination state based on response data"""
        if not response.json():  # Stop if no more data
            self._has_next_page = False
        else:
            self.current_time += self.interval_ms

    def update_request(self, request: Request) -> None:
        """Update the request parameters with the current time range"""
        if request.params is None:
            request.params = {}

        next_end_time = min(self.current_time + self.interval_ms, self.end_time)
        request.params['startTime'] = self.current_time
        request.params['endTime'] = next_end_time


In [ ]:
with open('aggtrades.txt','r') as data:
    data = data.read()
data = json.loads(data)

In [ ]:
len(data)

In [ ]:
data[0]['T']

In [ ]:
type(data[0]['T'])

In [ ]:
datetime.fromtimestamp(data[0]['T']/1000)

In [ ]:
df = pd.DataFrame(data=data)
df.info()

In [ ]:
with open('depth.txt','r') as depth:
    depth = depth.read()
depth = json.loads(depth)
len(depth)

In [ ]:
depth

In [ ]:
def read_file(file):
    with open(file,'r') as file_name:
        data = file_name.read()
    data = json.loads(data)
    print(len(data))
    print(data[0])
    #return data

In [ ]:
read_file('trades.txt')

In [ ]:
read_file('depth.txt')

In [ ]:
read_file('aggtrades.txt')

In [ ]:
import sys

In [ ]:
sys.version

In [ ]:
sys.executable

In [ ]:
!pip -V

In [ ]:
!pip uninstall pyarrow -y

In [ ]:
pip install --upgrade pyarrow dlt

In [4]:
pip -V

pip 24.2 from /Users/koray/opt/anaconda3/lib/python3.9/site-packages/pip (python 3.9)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip list

In [5]:
dlt.__version__

'1.8.1'

In [6]:
pyarrow.__version__

'19.0.1'

In [6]:
@dlt.resource(name="aggtrades", write_disposition="replace") #aggtrades will be the table name
def binance_api():
    client = RESTClient(
        base_url="https://data-api.binance.vision"
        ,paginator=TimeRangePaginator(
        start_time=1672531200000,  # Start time in milliseconds (e.g., 2023-01-01)
        end_time=1672617600000,    # End time in milliseconds (e.g., 2023-01-02)
        interval_ms=60 * 60 * 1000  # 1-hour interval
        )
        
    )

    for page in client.paginate("/api/v3/aggTrades?symbol=BTCUSDT"):
        yield page

In [7]:
# define new dlt pipeline
pipeline = dlt.pipeline(
    destination="duckdb", #database technology
    pipeline_name='binance', #database name in the destination
    dataset_name='aggtrade' #dataset name in the destination
)

# run the pipeline with the new resource
load_info = pipeline.run(binance_api, write_disposition="replace")
print(load_info)

# explore loaded data
pipeline.dataset(dataset_type="default").aggtrades.df()

Pipeline binance load step completed in 0.75 seconds
1 load package(s) were loaded to destination duckdb and into dataset aggtrade
The duckdb destination used duckdb:////Users/koray/Documents/GitHub/deng25/Project/binance.duckdb location to store data
Load package 1742980048.5908082 is LOADED and contains no failed jobs


,a,p,q,f,l,t,m,_dlt_load_id,_dlt_id
0,2054785120,16541.77000000,0.00257000,2408185868,2408185868,1672531200001,True,1742980048.5908082,a34qEL2xon6vIw
1,2054785121,16541.77000000,0.00177000,2408185869,2408185869,1672531200007,True,1742980048.5908082,s5CJEPzRoya44A
2,2054785122,16541.77000000,0.05700000,2408185870,2408185871,1672531200007,True,1742980048.5908082,Etqpt5x3O5Kf+w
3,2054785123,16541.77000000,0.00073000,2408185872,2408185872,1672531200008,True,1742980048.5908082,DwXQTfsoQb5SlA
4,2054785124,16542.40000000,0.00133000,2408185873,2408185873,1672531200009,True,1742980048.5908082,P6ueV9CjueaXFw
...,...,...,...,...,...,...,...,...,...
11995,2057168533,16602.56000000,0.00971000,2411276417,2411276417,1672614019348,True,1742980048.5908082,BcjBcwpDTDxE/g
11996,2057168534,16602.43000000,0.00098000,2411276418,2411276418,1672614019603,True,1742980048.5908082,OwJ4rVR4jMQ3MA
11997,2057168535,16602.56000000,0.00507000,2411276419,2411276419,1672614019603,True,1742980048.5908082,0MfnRMwxWPmGyQ
11998,2057168536,16602.55000000,0.03997000,2411276420,2411276420,1672614019663,True,1742980048.5908082,6XKZWGZVCK3q9w


In [9]:
dlt pipeline binance show

SyntaxError: invalid syntax (2801371419.py, line 1)

In [30]:
con = duckdb.connect('binance.duckdb')

In [31]:
con.sql('SELECT * from INFORMATION_SCHEMA.tables').df()

,table_catalog,table_schema,table_name,table_type,self_referencing_column_name,reference_generation,user_defined_type_catalog,user_defined_type_schema,user_defined_type_name,is_insertable_into,is_typed,commit_action,TABLE_COMMENT
0,binance,aggtrade,aggtrades,BASE TABLE,None,None,None,None,None,YES,NO,None,None
1,binance,aggtrade,_dlt_loads,BASE TABLE,None,None,None,None,None,YES,NO,None,None
2,binance,aggtrade,_dlt_pipeline_state,BASE TABLE,None,None,None,None,None,YES,NO,None,None
3,binance,aggtrade,_dlt_version,BASE TABLE,None,None,None,None,None,YES,NO,None,None


In [33]:
con.sql('SELECT * from aggtrade.aggtrades').df()

,a,p,q,f,l,t,m,_dlt_load_id,_dlt_id
0,3506862980,87100.00000000,0.02200000,4743236727,4743236727,1742906902016,True,1742906979.34092,MD8nn22OTDdxhA
1,3506862981,87100.00000000,0.02200000,4743236728,4743236728,1742906902016,True,1742906979.34092,HPowQuD8TnINYw
2,3506862982,87100.00000000,0.02600000,4743236729,4743236729,1742906902016,True,1742906979.34092,55Elv0VcGnKR+w
3,3506862983,87100.00000000,0.02200000,4743236730,4743236730,1742906902017,True,1742906979.34092,m9Zfht9lBVF44w
4,3506862984,87100.00000000,0.02200000,4743236731,4743236731,1742906902017,True,1742906979.34092,CjlSOLIk7GCtmw
...,...,...,...,...,...,...,...,...,...
495,3506863475,87099.17000000,0.00012000,4743237896,4743237896,1742906978136,True,1742906979.34092,RTO9vlFOrnCYsw
496,3506863476,87099.17000000,0.08355000,4743237897,4743237897,1742906978179,True,1742906979.34092,NMy5m46pJJ3Ftw
497,3506863477,87099.18000000,0.00559000,4743237898,4743237898,1742906979177,True,1742906979.34092,RHwKhrNlIREA/g
498,3506863478,87099.18000000,0.02786000,4743237899,4743237904,1742906979946,True,1742906979.34092,QTe6VBQO8c6vyg


In [ ]:
!curl 